In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import os
import zipfile
import requests
from tqdm import tqdm
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [3]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [5]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [6]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [7]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
#         print (tgt_mask.shape)
#         print (nopeak_mask.shape)
        tgt_mask = tgt_mask & nopeak_mask

        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [8]:
src_vocab_size = 32000
tgt_vocab_size = 32000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 20
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)


In [9]:
## Read training data

# Constants
DOWNLOAD_URL = "https://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/moses/en-es.txt.zip"
EXTRACT_DIR = "/Users/fangzhang/Desktop/build_transformer/en-es.txt"
ZIP_FILE = "en-es.txt.zip"
OUTPUT_FILE = "opensubtitles_en_es.tsv"

# Step 1: Read and clean the sentence pairs

src_file_path = os.path.join(EXTRACT_DIR, "OpenSubtitles.en-es.en")
tgt_file_path = os.path.join(EXTRACT_DIR, "OpenSubtitles.en-es.es")

print("Processing sentence pairs...")
with open(src_file_path, 'r', encoding='utf-8') as src_file, \
     open(tgt_file_path, 'r', encoding='utf-8') as tgt_file, \
     open(OUTPUT_FILE, 'w', encoding='utf-8') as out_file:
    for src_line, tgt_line in tqdm(zip(src_file, tgt_file), desc="Cleaning", unit="lines"):
        src_line = src_line.strip()
        tgt_line = tgt_line.strip()
        if src_line and tgt_line and len(src_line.split()) < 50 and len(tgt_line.split()) < 50:
            out_file.write(f"{src_line}\t{tgt_line}\n")

print(f"\nSaved cleaned sentence pairs to {OUTPUT_FILE} ✅")


# Step 2: Load the TSV file with error handling
df = pd.read_csv(
    'opensubtitles_en_es.tsv',
    sep='\t',
    header=None,
    names=['en', 'es'],
    engine='python',
    on_bad_lines='warn'  # Skips bad lines and warns about them
)

# Step 3: processing
# Drop rows with null values
df.dropna(inplace=True)

# Filter sentence pairs by length
df = df[df['en'].str.split().str.len().between(3, 50)]
df = df[df['es'].str.split().str.len().between(3, 50)]

# Remove duplicate pairs
df.drop_duplicates(inplace=True)

# Reset index
df.reset_index(drop=True, inplace=True)


# Step 4: Initialize tokenizers for English and Spanish
tokenizer_en = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer_es = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Tokenize the sentences
df['en_tokens'] = df['en'].apply(lambda x: tokenizer_en.encode(x, add_special_tokens=True))
df['es_tokens'] = df['es'].apply(lambda x: tokenizer_es.encode(x, add_special_tokens=True))



# Step 5: Split into training and temp (which will be further split into validation and test)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Split temp into validation and test
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)



# Example pandas Series
# Each value is a list of integers representing tokens

# Parameters
max_len = 20
pad_token = 0  # You can choose any value as the padding token

# Step 6: convert to tensors

def gen_tensor(s):

    # Pad or truncate each list
    padded = s.apply(lambda x: (x + [pad_token]*max_len)[:max_len])

    # Convert to tensor
    data = torch.tensor(padded.tolist(), dtype=torch.long)
    return data

src_data = gen_tensor(train_df['en_tokens'])
tgt_data = gen_tensor(train_df['es_tokens'])

src_test = gen_tensor(test_df['en_tokens'])
tgt_test = gen_tensor(test_df['es_tokens'])



Skipping line 589: '	' expected after '"'
Skipping line 1782: '	' expected after '"'
Skipping line 2129: '	' expected after '"'
Skipping line 2352: '	' expected after '"'
Skipping line 2353: '	' expected after '"'
Skipping line 2354: '	' expected after '"'
Skipping line 2580: '	' expected after '"'
Skipping line 2882: '	' expected after '"'
Skipping line 2883: '	' expected after '"'
Skipping line 2908: '	' expected after '"'
Skipping line 2910: '	' expected after '"'
Skipping line 2911: '	' expected after '"'
Skipping line 3355: '	' expected after '"'
Skipping line 3703: '	' expected after '"'
Skipping line 4442: '	' expected after '"'
Skipping line 4696: '	' expected after '"'
Skipping line 4758: '	' expected after '"'
Skipping line 4759: '	' expected after '"'
Skipping line 4760: '	' expected after '"'
Skipping line 4841: '	' expected after '"'
Skipping line 4842: '	' expected after '"'
Skipping line 4843: '	' expected after '"'
Skipping line 5357: '	' expected after '"'
Skipping lin

Skipping line 179920: '	' expected after '"'
Skipping line 180137: '	' expected after '"'
Skipping line 180326: '	' expected after '"'
Skipping line 181193: '	' expected after '"'
Skipping line 181194: '	' expected after '"'
Skipping line 181195: '	' expected after '"'
Skipping line 181200: '	' expected after '"'
Skipping line 181201: '	' expected after '"'
Skipping line 181202: '	' expected after '"'
Skipping line 181203: '	' expected after '"'
Skipping line 181206: '	' expected after '"'
Skipping line 181207: '	' expected after '"'
Skipping line 181208: '	' expected after '"'
Skipping line 181209: '	' expected after '"'
Skipping line 181232: '	' expected after '"'
Skipping line 181233: '	' expected after '"'
Skipping line 181604: '	' expected after '"'
Skipping line 181829: '	' expected after '"'
Skipping line 181973: '	' expected after '"'
Skipping line 182465: '	' expected after '"'
Skipping line 182497: '	' expected after '"'
Skipping line 182501: '	' expected after '"'
Skipping l

Skipping line 443702: '	' expected after '"'
Skipping line 444033: '	' expected after '"'
Skipping line 444248: '	' expected after '"'
Skipping line 444422: '	' expected after '"'
Skipping line 444425: '	' expected after '"'
Skipping line 444445: '	' expected after '"'
Skipping line 444843: '	' expected after '"'
Skipping line 445456: '	' expected after '"'
Skipping line 445468: '	' expected after '"'
Skipping line 446301: '	' expected after '"'
Skipping line 446532: '	' expected after '"'
Skipping line 446841: '	' expected after '"'
Skipping line 446912: '	' expected after '"'
Skipping line 446971: '	' expected after '"'
Skipping line 447784: '	' expected after '"'
Skipping line 449549: '	' expected after '"'
Skipping line 449550: '	' expected after '"'
Skipping line 449704: '	' expected after '"'
Skipping line 449889: '	' expected after '"'
Skipping line 449902: '	' expected after '"'
Skipping line 449903: '	' expected after '"'
Skipping line 449904: '	' expected after '"'
Skipping l

In [14]:
src_data.shape

torch.Size([284734, 20])

In [89]:
transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [10]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

Transformer(
  (encoder_embedding): Embedding(32000, 512)
  (decoder_embedding): Embedding(32000, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x DecoderLayer(

In [16]:



dataset = TensorDataset(src_data, tgt_data)

# Create DataLoader with batching
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Usage in training loop
for batch in loader:
    src_batch, tgt_batch = batch  # both are shape (batch_size, seq_len)
    # Feed into transformer model here
    
    for epoch in range(2):
        optimizer.zero_grad()
        output = transformer(src_batch, tgt_batch[:, :-1])
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_batch[:, 1:].contiguous().view(-1))
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")


Epoch: 1, Loss: 7.527031421661377
Epoch: 2, Loss: 7.415209770202637
Epoch: 1, Loss: 7.341157913208008
Epoch: 2, Loss: 7.220810413360596
Epoch: 1, Loss: 7.134400844573975
Epoch: 2, Loss: 7.037831783294678
Epoch: 1, Loss: 6.976173400878906
Epoch: 2, Loss: 6.912863731384277
Epoch: 1, Loss: 6.910312652587891
Epoch: 2, Loss: 6.8360209465026855
Epoch: 1, Loss: 7.260377407073975
Epoch: 2, Loss: 7.207149982452393
Epoch: 1, Loss: 6.904798984527588
Epoch: 2, Loss: 6.843395233154297
Epoch: 1, Loss: 6.905842304229736
Epoch: 2, Loss: 6.85593843460083
Epoch: 1, Loss: 6.582320213317871
Epoch: 2, Loss: 6.522158145904541
Epoch: 1, Loss: 6.829554080963135
Epoch: 2, Loss: 6.777528285980225
Epoch: 1, Loss: 6.763797283172607
Epoch: 2, Loss: 6.6898512840271
Epoch: 1, Loss: 6.627096176147461
Epoch: 2, Loss: 6.544992923736572
Epoch: 1, Loss: 6.64716911315918
Epoch: 2, Loss: 6.609177112579346
Epoch: 1, Loss: 6.538753509521484
Epoch: 2, Loss: 6.488523960113525
Epoch: 1, Loss: 6.682265281677246
Epoch: 2, Loss: 6

Epoch: 1, Loss: 5.289556980133057
Epoch: 2, Loss: 5.221793174743652
Epoch: 1, Loss: 5.270346164703369
Epoch: 2, Loss: 5.23259973526001
Epoch: 1, Loss: 5.708643436431885
Epoch: 2, Loss: 5.644121170043945
Epoch: 1, Loss: 5.420557498931885
Epoch: 2, Loss: 5.380421161651611
Epoch: 1, Loss: 5.38323450088501
Epoch: 2, Loss: 5.323607921600342
Epoch: 1, Loss: 5.6000752449035645
Epoch: 2, Loss: 5.534358024597168
Epoch: 1, Loss: 5.094754695892334
Epoch: 2, Loss: 5.0509514808654785
Epoch: 1, Loss: 5.33390998840332
Epoch: 2, Loss: 5.31030797958374
Epoch: 1, Loss: 5.346582412719727
Epoch: 2, Loss: 5.284322738647461
Epoch: 1, Loss: 5.3942790031433105
Epoch: 2, Loss: 5.387412071228027
Epoch: 1, Loss: 5.200820446014404
Epoch: 2, Loss: 5.140238285064697
Epoch: 1, Loss: 5.432964324951172
Epoch: 2, Loss: 5.3962812423706055
Epoch: 1, Loss: 5.582995891571045
Epoch: 2, Loss: 5.526856899261475
Epoch: 1, Loss: 5.60748815536499
Epoch: 2, Loss: 5.573823928833008
Epoch: 1, Loss: 5.389953136444092
Epoch: 2, Loss:

Epoch: 2, Loss: 4.902703285217285
Epoch: 1, Loss: 5.108667373657227
Epoch: 2, Loss: 5.04913330078125
Epoch: 1, Loss: 5.289192199707031
Epoch: 2, Loss: 5.2305006980896
Epoch: 1, Loss: 4.965766429901123
Epoch: 2, Loss: 4.916149139404297
Epoch: 1, Loss: 4.870975494384766
Epoch: 2, Loss: 4.827175617218018
Epoch: 1, Loss: 5.271073341369629
Epoch: 2, Loss: 5.211513519287109
Epoch: 1, Loss: 5.266929626464844
Epoch: 2, Loss: 5.213219165802002
Epoch: 1, Loss: 5.343790054321289
Epoch: 2, Loss: 5.322932720184326
Epoch: 1, Loss: 5.171017646789551
Epoch: 2, Loss: 5.108606815338135
Epoch: 1, Loss: 4.952931880950928
Epoch: 2, Loss: 4.913938999176025
Epoch: 1, Loss: 5.376008033752441
Epoch: 2, Loss: 5.3405609130859375
Epoch: 1, Loss: 5.3449578285217285
Epoch: 2, Loss: 5.283731460571289
Epoch: 1, Loss: 5.201436996459961
Epoch: 2, Loss: 5.168420791625977
Epoch: 1, Loss: 5.270475387573242
Epoch: 2, Loss: 5.21876859664917
Epoch: 1, Loss: 5.3214898109436035
Epoch: 2, Loss: 5.268990993499756
Epoch: 1, Loss:

Epoch: 1, Loss: 4.856122970581055
Epoch: 2, Loss: 4.80445671081543
Epoch: 1, Loss: 5.001396656036377
Epoch: 2, Loss: 4.970744609832764
Epoch: 1, Loss: 4.7276611328125
Epoch: 2, Loss: 4.668872833251953
Epoch: 1, Loss: 4.967513084411621
Epoch: 2, Loss: 4.913802623748779
Epoch: 1, Loss: 5.124876499176025
Epoch: 2, Loss: 5.094095706939697
Epoch: 1, Loss: 4.934475421905518
Epoch: 2, Loss: 4.875233173370361
Epoch: 1, Loss: 4.7463555335998535
Epoch: 2, Loss: 4.725415229797363
Epoch: 1, Loss: 4.593967437744141
Epoch: 2, Loss: 4.564427852630615
Epoch: 1, Loss: 4.903465747833252
Epoch: 2, Loss: 4.864099502563477
Epoch: 1, Loss: 4.826308250427246
Epoch: 2, Loss: 4.767971515655518
Epoch: 1, Loss: 4.758874416351318
Epoch: 2, Loss: 4.7173380851745605
Epoch: 1, Loss: 5.050273895263672
Epoch: 2, Loss: 5.003370761871338
Epoch: 1, Loss: 4.922666549682617
Epoch: 2, Loss: 4.857700347900391
Epoch: 1, Loss: 4.99091911315918
Epoch: 2, Loss: 4.947790622711182
Epoch: 1, Loss: 5.053999900817871
Epoch: 2, Loss: 

Epoch: 2, Loss: 4.912747383117676
Epoch: 1, Loss: 4.676827430725098
Epoch: 2, Loss: 4.6146321296691895
Epoch: 1, Loss: 4.931901931762695
Epoch: 2, Loss: 4.899764537811279
Epoch: 1, Loss: 4.721207141876221
Epoch: 2, Loss: 4.675357818603516
Epoch: 1, Loss: 4.570675373077393
Epoch: 2, Loss: 4.5432562828063965
Epoch: 1, Loss: 5.185649394989014
Epoch: 2, Loss: 5.124850749969482
Epoch: 1, Loss: 4.5594401359558105
Epoch: 2, Loss: 4.503262519836426
Epoch: 1, Loss: 4.791454792022705
Epoch: 2, Loss: 4.735711097717285
Epoch: 1, Loss: 4.905610084533691
Epoch: 2, Loss: 4.854436874389648
Epoch: 1, Loss: 4.430503845214844
Epoch: 2, Loss: 4.366552829742432
Epoch: 1, Loss: 4.438321113586426
Epoch: 2, Loss: 4.401785850524902
Epoch: 1, Loss: 4.672057628631592
Epoch: 2, Loss: 4.624536037445068
Epoch: 1, Loss: 4.696045398712158
Epoch: 2, Loss: 4.652597904205322
Epoch: 1, Loss: 4.8215012550354
Epoch: 2, Loss: 4.7721757888793945
Epoch: 1, Loss: 4.869983673095703
Epoch: 2, Loss: 4.827706813812256
Epoch: 1, Lo

...

Epoch: 2, Loss: 3.64658784866333
Epoch: 1, Loss: 3.5173087120056152
Epoch: 2, Loss: 3.4457826614379883
Epoch: 1, Loss: 3.8459675312042236
Epoch: 2, Loss: 3.7987070083618164
Epoch: 1, Loss: 3.715845823287964
Epoch: 2, Loss: 3.6970138549804688
Epoch: 1, Loss: 3.6027185916900635
Epoch: 2, Loss: 3.55666446685791
Epoch: 1, Loss: 3.8738651275634766
Epoch: 2, Loss: 3.807285785675049
Epoch: 1, Loss: 4.050968170166016
Epoch: 2, Loss: 3.971153497695923
Epoch: 1, Loss: 3.6055257320404053
Epoch: 2, Loss: 3.5325255393981934
Epoch: 1, Loss: 3.611656665802002
Epoch: 2, Loss: 3.557361364364624
Epoch: 1, Loss: 3.546855926513672
Epoch: 2, Loss: 3.4658687114715576
Epoch: 1, Loss: 3.6951849460601807
Epoch: 2, Loss: 3.6549463272094727
Epoch: 1, Loss: 3.618985652923584
Epoch: 2, Loss: 3.582848310470581
Epoch: 1, Loss: 3.5492804050445557
Epoch: 2, Loss: 3.4957926273345947
Epoch: 1, Loss: 3.4622104167938232
Epoch: 2, Loss: 3.393993854522705
Epoch: 1, Loss: 3.9040656089782715
Epoch: 2, Loss: 3.8294057846069336

Epoch: 2, Loss: 3.8444299697875977
Epoch: 1, Loss: 3.6927433013916016
Epoch: 2, Loss: 3.6275577545166016
Epoch: 1, Loss: 3.4708354473114014
Epoch: 2, Loss: 3.399589776992798
Epoch: 1, Loss: 3.674837589263916
Epoch: 2, Loss: 3.619652509689331
Epoch: 1, Loss: 3.5583395957946777
Epoch: 2, Loss: 3.4891090393066406
Epoch: 1, Loss: 3.551422595977783
Epoch: 2, Loss: 3.5064945220947266
Epoch: 1, Loss: 4.035143852233887
Epoch: 2, Loss: 3.988542079925537
Epoch: 1, Loss: 3.5418927669525146
Epoch: 2, Loss: 3.490800142288208
Epoch: 1, Loss: 3.4347777366638184
Epoch: 2, Loss: 3.3518121242523193
Epoch: 1, Loss: 3.6214053630828857
Epoch: 2, Loss: 3.5709729194641113
Epoch: 1, Loss: 3.59371280670166
Epoch: 2, Loss: 3.534214735031128
Epoch: 1, Loss: 3.8296613693237305
Epoch: 2, Loss: 3.811161994934082
Epoch: 1, Loss: 3.7470927238464355
Epoch: 2, Loss: 3.704113006591797
Epoch: 1, Loss: 3.240614175796509
Epoch: 2, Loss: 3.174739122390747
Epoch: 1, Loss: 3.5564112663269043
Epoch: 2, Loss: 3.4813973903656006

Epoch: 2, Loss: 3.5557236671447754
Epoch: 1, Loss: 3.5805342197418213
Epoch: 2, Loss: 3.5314269065856934
Epoch: 1, Loss: 3.419729471206665
Epoch: 2, Loss: 3.3732094764709473
Epoch: 1, Loss: 3.8957316875457764
Epoch: 2, Loss: 3.8394343852996826
Epoch: 1, Loss: 3.4179556369781494
Epoch: 2, Loss: 3.3913772106170654
Epoch: 1, Loss: 3.847749710083008
Epoch: 2, Loss: 3.8205831050872803
Epoch: 1, Loss: 3.686664342880249
Epoch: 2, Loss: 3.649380683898926
Epoch: 1, Loss: 3.671210289001465
Epoch: 2, Loss: 3.579653739929199
Epoch: 1, Loss: 3.4887776374816895
Epoch: 2, Loss: 3.4166207313537598
Epoch: 1, Loss: 3.6814348697662354
Epoch: 2, Loss: 3.644228935241699
Epoch: 1, Loss: 3.509887456893921
Epoch: 2, Loss: 3.4448354244232178
Epoch: 1, Loss: 3.7247226238250732
Epoch: 2, Loss: 3.685195207595825
Epoch: 1, Loss: 3.751084327697754
Epoch: 2, Loss: 3.7174456119537354
Epoch: 1, Loss: 3.886507511138916
Epoch: 2, Loss: 3.806072950363159
Epoch: 1, Loss: 3.515432119369507
Epoch: 2, Loss: 3.452006340026855

Epoch: 2, Loss: 3.3034327030181885
Epoch: 1, Loss: 3.5229713916778564
Epoch: 2, Loss: 3.4443624019622803
Epoch: 1, Loss: 3.9406232833862305
Epoch: 2, Loss: 3.8890798091888428
Epoch: 1, Loss: 3.467158794403076
Epoch: 2, Loss: 3.387998104095459
Epoch: 1, Loss: 3.6659772396087646
Epoch: 2, Loss: 3.59980845451355
Epoch: 1, Loss: 3.294703722000122
Epoch: 2, Loss: 3.229099988937378
Epoch: 1, Loss: 3.636596202850342
Epoch: 2, Loss: 3.6240310668945312
Epoch: 1, Loss: 3.641580581665039
Epoch: 2, Loss: 3.5793144702911377
Epoch: 1, Loss: 3.734959602355957
Epoch: 2, Loss: 3.68233323097229
Epoch: 1, Loss: 4.091590404510498
Epoch: 2, Loss: 4.035525321960449
Epoch: 1, Loss: 3.716032028198242
Epoch: 2, Loss: 3.6566059589385986
Epoch: 1, Loss: 3.7213001251220703
Epoch: 2, Loss: 3.667464017868042
Epoch: 1, Loss: 3.772998571395874
Epoch: 2, Loss: 3.709501266479492
Epoch: 1, Loss: 3.3685600757598877
Epoch: 2, Loss: 3.336576223373413
Epoch: 1, Loss: 3.7719779014587402
Epoch: 2, Loss: 3.697596549987793
Epoc

Epoch: 2, Loss: 3.2973811626434326
Epoch: 1, Loss: 3.528472661972046
Epoch: 2, Loss: 3.424164056777954
Epoch: 1, Loss: 3.821993589401245
Epoch: 2, Loss: 3.77785325050354
Epoch: 1, Loss: 3.73510479927063
Epoch: 2, Loss: 3.6463921070098877
Epoch: 1, Loss: 3.65343976020813
Epoch: 2, Loss: 3.609405279159546
Epoch: 1, Loss: 3.7256202697753906
Epoch: 2, Loss: 3.652801036834717
Epoch: 1, Loss: 3.987642765045166
Epoch: 2, Loss: 3.941910743713379
Epoch: 1, Loss: 3.909152030944824
Epoch: 2, Loss: 3.860456705093384
Epoch: 1, Loss: 3.557239294052124
Epoch: 2, Loss: 3.4798905849456787
Epoch: 1, Loss: 3.755885124206543
Epoch: 2, Loss: 3.705014944076538
Epoch: 1, Loss: 3.466625928878784
Epoch: 2, Loss: 3.4081151485443115
Epoch: 1, Loss: 3.276688575744629
Epoch: 2, Loss: 3.1778995990753174
Epoch: 1, Loss: 4.011191368103027
Epoch: 2, Loss: 3.9425277709960938
Epoch: 1, Loss: 3.394745111465454
Epoch: 2, Loss: 3.361379623413086
Epoch: 1, Loss: 3.421466112136841
Epoch: 2, Loss: 3.3961825370788574
Epoch: 1,

In [25]:
transformer.eval()

src_test = gen_tensor(test_df.iloc[:1900,2])
tgt_test = gen_tensor(test_df.iloc[:1900,3])

with torch.no_grad():

    val_output = transformer(src_test, tgt_test[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), tgt_test[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")

Validation Loss: 3.561875820159912
